# Do the Doodle: Predicting "Quick, Draw!" drawings based on player's first drawing stroke

EC Corro | SS Garcia | J Gonzales | CR Patalud 

<div><span style="background-color: #1F773B; padding-top: 100px; padding-right: 20px; padding-bottom: 50px; padding-left: 20px; color: #F2FBF5; font-size: 20px; font-weight: bold">DATA PREPROCESSING</span></div>

# Dask Distributed Setup and Data Preprocessing 

In [1]:
import dask.bag as db
import dask.array as da
import dask.dataframe as dd
# from dask.distributed import Client


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings("ignore")


### Preselected Doodle Dataset are Stored in an S3 bucket `s3://bdccdoodle/drawings_csv`

The dataset size used for this project is 53.1 GB.\
This is composed of 82 csv files ranging from 278.8 MB to 1.3 GB

In [5]:
%%bash
aws s3 ls s3://bdccdoodle/drawings_csv/ --human-readable --summarize 

2021-01-23 12:10:19    0 Bytes 
2021-01-23 12:11:44  383.5 MiB ant.csv
2021-01-23 12:11:44  464.2 MiB apple.csv
2021-01-23 12:11:44  454.7 MiB arm.csv
2021-01-23 12:11:46  667.4 MiB asparagus.csv
2021-01-23 12:11:46  914.6 MiB banana.csv
2021-01-23 12:11:44  618.5 MiB bat.csv
2021-01-23 12:11:46  627.2 MiB bear.csv
2021-01-23 12:11:44  579.2 MiB bee.csv
2021-01-23 12:11:44  567.8 MiB bird.csv
2021-01-23 12:11:46  655.9 MiB blackberry.csv
2021-01-23 12:11:44  375.4 MiB blueberry.csv
2021-01-23 12:11:46  873.4 MiB brain.csv
2021-01-23 12:11:46  633.0 MiB broccoli.csv
2021-01-23 12:11:44  550.3 MiB butterfly.csv
2021-01-23 12:11:44  498.3 MiB carrot.csv
2021-01-23 12:11:44  565.5 MiB cat.csv
2021-01-23 12:11:46  853.5 MiB cow.csv
2021-01-23 12:11:46  718.5 MiB crab.csv
2021-01-23 12:11:46  740.1 MiB crocodile.csv
2021-01-23 12:11:46  778.5 MiB dog.csv
2021-01-23 12:11:44  535.3 MiB dolphin.csv
2021-01-23 12:11:47    1.3 GiB duck.csv
2021-01-23 12:11:44  288.5 MiB ear.csv
2021-01-23 12:11:

### Loading the Data

|column name|Data Type|Description|
|:--|:--|:--|
|countrycode|String|Country Code where the drawing originated|
|drawing|Array|Drawing array representing the drawing's stroke coordinates and time|
|key_id|String|Unique key ID of the drawing|
|recognized|Boolean|Boolean identification whether the drawing was successfuly recognized by the AI|
|timestamp|Time Object|Timestamp when the object was drawn|
|word|String|The Object Being Drawn by the Participant|

In [5]:
df = dd.read_csv('s3://bdccdoodle/drawings_csv/*.csv', 
                  error_bad_lines=False,
                  storage_options={'anon':True}, 
                  dtype={'drawing': 'object'})

In [6]:
df.tail(2)

countrycode  \
5880          GB   
5881          US   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Data Preprocessing

### Funtions to be used for Data Preprocessing

In [16]:
def get_num_strokes(data):
    """Return num_strokes set of stroke elements x, y, t coordinates in string
    format.
    """
    data = data[1:-1]
    x = data.split(']], ')
    x[-1] = x[-1][:-2]
    x = [st + ']]' for st in x]  
    return len(x)

def get_strokes(data, num_strokes=1):
    """Return num_strokes set of stroke elements x, y, t coordinates in string
    format.
    """
    data = data[1:-1]
    x = data.split(']], ')
    x[-1] = x[-1][:-2]
    x = [st + ']]' for st in x]    
    return x[:num_strokes]

def to_nparray(lst):
    """Convert string-formatted list of integers as np.array of integers."""
    lst = re.sub("[\[\]\s]", "", lst)
    lst = lst.split(',')
    lst = np.array([int(x) for x in lst])    
    return lst

def stroke_to_nparray(stroke):
    """Return as numpy array the list of stroke elements."""
    coord_pattern = re.compile(r"\[[0-9,\s]*\]")
    coordinates = coord_pattern.findall(stroke)
    coordinates = np.array([to_nparray(coord) for coord in coordinates])   
    return coordinates

def allstrokes_to_nparray(strokes):
    """Return as np.array the list of strokes."""
    return np.array([stroke_to_nparray(elem) for elem in strokes])

def get_final_n_strokes(data, num_strokes=1):
    """Return the completely np.array transformed strokes."""
    lst = get_strokes(data, num_strokes=num_strokes)    
    return allstrokes_to_nparray(lst)

def limit_stroke_length(data, min_len=50, max_len=70):
    """Return data with stroke length specified."""    
    data1 = data.copy()
    
    # Select rows with stroke length limits specified.
    data1 = data1.loc[(data1.stroke_length >= min_len) &
                      (data1.stroke_length <= max_len)]
    data1 = data1.loc[(data1.stroke_length >= min_len) & 
                      (data1.stroke_length <= max_len)]
    
    # Get only first `stroke_length` points.
    data1.x = data1.x.apply(lambda x : x[:min_len])
    data1.y = data1.y.apply(lambda x : x[:min_len])
    
    return data1

#### Get the number of stroke

In [8]:
df['stroke_num'] = df.drawing.apply(lambda x: get_num_strokes(x))

#### Get the first stroke

In [9]:
df['stroke1'] = df.drawing.apply(lambda x: get_final_n_strokes(x)[0])

#### Get only valid strokes (with x and y)

In [10]:
df = df[df['stroke1'].map(len) > 1].copy()

#### Add stroke length column

In [11]:
df["stroke_length"] = df.stroke1.apply(lambda x: len(x[0]))

In [12]:
stats = df.stroke_length.describe().compute()

In [13]:
mean = int(stats['mean'])
std = int(stats['std'])
print('mean: ', mean)
print('std: ', std)

mean:  86
std:  74


#### Separate x and y

In [14]:
df['x'] = df.stroke1.apply(lambda x: x[0])
df['y'] = df.stroke1.apply(lambda x: x[1])

#### Select rows with `stroke_length` identified.

In [15]:
df = limit_stroke_length(df, min_len=mean, max_len=np.inf)

#### Make each x and y coordinate as columns and final formatting

In [17]:
# creat a copy of the main df
df1 = df.copy()

In [18]:
# drop unnecessary columns
df1 = df1.drop(['drawing', 'key_id', 'timestamp', 'stroke1'],
               axis=1).dropna().reset_index(drop=True)

In [19]:
df1.columns

Index(['countrycode', 'recognized', 'word', 'stroke_num', 'stroke_length', 'x',
       'y'],
      dtype='object')

In [21]:
df1 = df1.compute()

In [214]:
# Separate x stroke points
x_stroke = df1.x.apply(pd.Series).add_prefix('x_')

In [215]:
df1 = pd.merge(df1, x_stroke, left_index=True, right_index=True)

In [217]:
# Separate y stroke points
y_stroke = df1.y.apply(pd.Series).add_prefix('y_')

In [218]:
df1 = pd.merge(df1, y_stroke, left_index=True, right_index=True)

In [220]:
# Drop x, y columns
df1 = df1.drop(['x', 'y'], axis=1).dropna().reset_index(drop=True)

In [221]:
# Convert `recognized` values to integer.
df1.recognized = df1.recognized.apply(lambda x : int(x))

In [222]:
# Final pd DataFrame
df1

,countrycode,recognized,word,stroke_num,stroke_length,x_0,x_1,x_2,x_3,x_4,...,y_76,y_77,y_78,y_79,y_80,y_81,y_82,y_83,y_84,y_85
0,US,1,ant,11,106,970,965,960,955,950,...,477,476,475,472,469,466,463,460,455,450
1,IT,0,ant,2,682,1460,1456,1453,1449,1446,...,1609,1621,1634,1643,1656,1673,1690,1707,1724,1737
2,US,1,ant,7,95,344,339,332,327,321,...,193,193,193,193,193,193,193,193,193,195
3,ID,1,ant,12,299,707,704,699,694,689,...,292,287,282,277,276,276,280,285,291,296
4,PT,1,ant,8,86,898,893,885,880,871,...,350,350,350,350,350,350,350,350,350,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3853934,GB,1,zebra,6,200,369,358,352,344,339,...,385,380,372,361,352,344,336,333,338,346
3853935,RU,1,zebra,22,264,383,389,395,401,410,...,489,489,489,490,491,491,491,491,491,491
3853936,NL,0,zebra,14,262,490,485,480,473,467,...,391,401,411,419,425,434,438,438,438,438
3853937,US,1,zebra,7,333,296,289,284,276,268,...,483,474,462,450,436,422,407,393,378,359


### Save Preprocessed Dask DataFrame Allocated S3 Bucket

#### Partition Dask DataFrame to distribute the size 

In [228]:
# Convert pd DataFrame to Dask DataFrame with Repartition
ddf = dd.from_pandas(df1, npartitions=4)

#### Save Dask DataFrame to CSV  File to allocated S3 Bucket

In [230]:
ddf.to_csv('s3://bdcc-doodle/draw_processed_csv/draw_processed_*.csv', 
           index=False)

['bdcc-doodle/draw_processed_csv/draw_processed_0.csv',
 'bdcc-doodle/draw_processed_csv/draw_processed_1.csv',
 'bdcc-doodle/draw_processed_csv/draw_processed_2.csv',
 'bdcc-doodle/draw_processed_csv/draw_processed_3.csv']

#### Validate if processed data is saved to the S3 Bucket `s3://bdcc-doodle/draw_processed_csv/`

In [231]:
%%bash
aws s3 ls s3://bdcc-doodle/draw_processed_csv/ --human-readable --summarize 

2021-01-25 10:33:01  654.7 MiB draw_processed_0.csv
2021-01-25 10:33:01  654.1 MiB draw_processed_1.csv
2021-01-25 10:33:01  654.3 MiB draw_processed_2.csv
2021-01-25 10:33:01  654.8 MiB draw_processed_3.csv

Total Objects: 4
   Total Size: 2.6 GiB


---